# Graded Assignment 3: 9 to 5

Time to show off your SQL skills! For each question, copy the SQL query you used and make note of the answer.

## The Dataset

For this assignment, you will be using the Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results which can be found on [Kaggle](https://www.kaggle.com/datasets/bls/employment).

## Business Issue

You are working for the Bureau of Labor Statistics with the United States government and have been approached by your boss with an important meeting request. You have been asked by your supervisor to meet with Dolly Parton whose nonprofit is looking to shed light on the state of employment in the United States. As part of the 9 to 5 project, their research is focused on production and nonsupervisory employees and how those employees fare compared to all employees in the United States. While the data the BLS collects from the CES is publicly available, Dolly Parton and her colleagues need your assistance navigating the thousands of rows in each table in LaborStatisticsDB.

## About the Dataset

This dataset comes directly from the Bureau of Labor Statistics’ Current Employment Survey (CES). Here are some things you need to know:

1. The industry table contains an NAICS code. This is different from the industry code. NAICS stands for North American Industry Classification System.
1. Series ID is composed of multiple different codes. CES stands for Current Employment Survey, the name of the survey which collected the data. The industry code as specified by the BLS and the data type code as specified in the datatype table.

## Set Up

To connect to the database, use the same connection info used during the SQL lessons. 

For the assignment, we will be using `LaborStatisticsDB`.

## Database Exploration

To start with, let’s get to know the database further.

1. Use this space to make note of each table in the database, the columns within each table, each column’s data type, and how the tables are connected. You can write this down or draw a diagram. Whatever method helps you get an understanding of what is going on with `LaborStatisticsDB`.
   
   To add a photo, diagram or document to your file, drop the file into the folder that holds this notebook.  Use the link button to the right of the  </> symbol in the gray part of this cell, the link is just the name of your file.

In [ ]:

--- LaborStatisticsDB Schema Analysis

The `LaborStatisticsDB` is organized as a **Star Schema**. 
This design features central "Fact" tables 
(which store the actual numbers) connected to several
 "Dimension" or "Lookup" tables (which provide context).

1. Fact Tables: `dbo.annual_2016`, `dbo.january_2017`
2. Dimension/Lookup Tables: `dbo.series`, `dbo.industry`, `dbo.datatype`, `dbo.supersector`, `dbo.seasonal`, `dbo.period`, `dbo.footnote`

 
-- How the Tables are Connected

The database does not use explicit `FOREIGN KEY` constraints. 

Instead, the tables are linked through **implicit logical connections** based on matching column names.

1.  Fact to Dimension:
 The "Fact" tables (`dbo.annual_2016` and `dbo.january_2017`) 
link to the central `dbo.series` table using `series_id`.
    * `dbo.annual_2016.series_id` $\rightarrow$ `dbo.series.series_id`
    * `dbo.january_2017.series_id` $\rightarrow$ `dbo.series.series_id`

2.  Dimension to Lookups: 

The central `dbo.series` table then links out to all the other "Lookup" tables to get human-readable names.
    * `dbo.series.industry_code` $\rightarrow$ `dbo.industry.industry_code`
    * `dbo.series.data_type_code` $\rightarrow$ `dbo.datatype.data_type_code`
    * `dbo.series.supersector_code` $\rightarrow$ `dbo.supersector.supersector_code`
    * `dbo.series.seasonal` $\rightarrow$ `dbo.seasonal.seasonal_code`

2. What is the datatype for women employees?


In [ ]:

-- Sql query


SELECT 
    data_type_code, 
    data_type_text
FROM 
    dbo.datatype
WHERE 
    data_type_text LIKE '%WOMEN EMPLOYEES%';

    data_type_code,                 data_type_text
      10                             Women Employees
      39                             Women Employees to all employees ratio

3. What is the series id for  women employees in the commercial banking industry in the financial activities supersector?

In [ ]:


series_id       series_title 

CES5552211010    Women employees
CEU5552211010    Women employees   

## Aggregate Your Friends and Code some SQL

Put together the following:

1. How many employees were reported in 2016 in all industries? Round to the nearest whole number.

In [ ]:

USE LaborStatisticsDB;

SELECT 
    ROUND(SUM(a.value), 0) AS total_employees_2016
FROM 
    annual_2016 AS a
JOIN 
    series AS s
    ON a.series_id = s.series_id
WHERE 
    s.data_type_code = '01';  -- '01' usually represents "All Employees"

    -- answer

-- total_employees_2016
--------------------
-- 2340612             
-- ((1 row affected))


2. How many women employees were reported in 2016 in all industries? Round to the nearest whole number. 

In [ ]:

USE LaborStatisticsDB;

SELECT 
    ROUND(SUM(a.value), 0) AS total_women_employees_2016
FROM 
    annual_2016 AS a
JOIN 
    series AS s
    ON a.series_id = s.series_id
WHERE 
    s.data_type_code = '06';  -- '06' typically represents "Women Employees"


3. How many production/nonsupervisory employees were reported in 2016? Round to the nearest whole number. 

In [ ]:

USE LaborStatisticsDB;

SELECT 
    ROUND(SUM(a.value), 0) AS total_production_nonsupervisory_employees_2016
FROM 
    annual_2016 AS a
JOIN 
    series AS s
    ON a.series_id = s.series_id
JOIN 
    datatype AS d
    ON s.data_type_code = d.data_type_code
WHERE 
    d.data_type_text = 'PRODUCTION AND NONSUPERVISORY EMPLOYEES';

-- answer
-- total_production_nonsupervisory_employees_2016
----------------------------------------------
1263650     
--
--                           ((1 row affected))

4. In January 2017, what is the average weekly hours worked by production and nonsupervisory employees across all industries?

In [ ]:
USE LaborStatisticsDB;
GO

SELECT 
    ROUND(AVG(m.value), 2) AS avg_weekly_hours_prod_nonsupervisory_jan_2017
FROM january_2017 AS m
JOIN series AS s 
    ON m.series_id = s.series_id
JOIN datatype AS d 
    ON s.data_type_code = d.data_type_code
WHERE 
    d.data_type_text = 'AVERAGE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES';

    -- answer:
--     Result Set 1-0
========================================
-- avg_weekly_hours_prod_nonsupervisory_jan_2017
---------------------------------------------
36.06  
--                                      ((1 row affected))




5. What is the total weekly payroll for production and nonsupervisory employees across all industries in January 2017? Round to the nearest penny.

In [ ]:

USE LaborStatisticsDB;
GO

SELECT 
    '$' + FORMAT(ROUND(SUM(m.value), 2), 'N2') AS total_weekly_payroll_prod_nonsupervisory_jan_2017
FROM january_2017 AS m
JOIN series AS s
    ON m.series_id = s.series_id
JOIN datatype AS d
    ON s.data_type_code = d.data_type_code
WHERE d.data_type_text = 'AGGREGATE WEEKLY PAYROLLS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES';


-- answer
-- Result Set 1-0
========================================
-- total_weekly_payroll_prod_nonsupervisory_jan_2017
-------------------------------------------------
$1,838,753,220.00     
--                           ((1 row affected))


6. In January 2017, for which industry was the average weekly hours worked by production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:

 -- Query for Highest Average Weekly Hours
SELECT TOP 1 
    i.industry_name,
    j.value AS avg_weekly_hours
FROM
    dbo.january_2017 AS j
JOIN
    dbo.series AS s ON j.series_id = s.series_id
JOIN
    dbo.industry AS i ON s.industry_code = i.industry_code
WHERE
    s.data_type_code = '07'
    AND i.industry_name NOT LIKE '%Total%'
ORDER BY
    j.value DESC; -- HIGHEST


--- Query for Lowest Average Weekly Hours ---

SELECT TOP 1 
    i.industry_name,
    j.value AS avg_weekly_hours
FROM
    dbo.january_2017 AS j
JOIN
    dbo.series AS s ON j.series_id = s.series_id
JOIN
    dbo.industry AS i ON s.industry_code = i.industry_code
WHERE
    s.data_type_code = '07'
    AND i.industry_name NOT LIKE '%Total%' 
ORDER BY
    j.value ASC; -- LOWEST

    --industry_name	avg_weekly_hours
    -- Motor vehicle power train components	49.8
--industry_name	avg_weekly_hours

--Fitness and recreational sports centers	16.7

7. In January 2021, for which industry was the total weekly payroll for production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:

USE LaborStatisticsDB;
GO

SELECT 
    i.industry_name,
    ROUND(SUM(m.value), 2) AS total_weekly_payroll_prod_nonsupervisory_jan_2017
FROM january_2017 AS m
JOIN series AS s
    ON m.series_id = s.series_id
JOIN datatype AS d
    ON s.data_type_code = d.data_type_code
JOIN industry AS i
    ON s.industry_code = i.industry_code
WHERE d.data_type_text = 'AGGREGATE WEEKLY PAYROLLS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES'
GROUP BY i.industry_name
ORDER BY total_weekly_payroll_prod_nonsupervisory_jan_2017 DESC;

## Join in on the Fun

Time to start joining! You can choose the type of join you use, just make sure to make a  note!

1. Join `annual_2016` with `series` on `series_id`. We only want the data in the `annual_2016` table to be included in the result.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *


-- Uncomment the line below when you are ready to run the query, leaving it as your last!

USE LaborStatisticsDB;
GO

-- Join annual_2016 with series
SELECT TOP 50 *
FROM annual_2016 AS a
LEFT JOIN series AS s
    ON a.series_id = s.series_id
ORDER BY a.id;

-- ORDER BY id


Commands completed successfully.

Total execution time: 00:00:00.019

2. Join `series` and `datatype` on `data_type_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *


-- Uncomment the line below when you are ready to run the query, leaving it as your last!

USE LaborStatisticsDB;
GO

-- Join series and datatype on data_type_code
SELECT TOP 50 *
FROM series AS s
INNER JOIN datatype AS d
    ON s.data_type_code = d.data_type_code
ORDER BY s.series_id;

-- ORDER BY id

3. Join `series` and `industry` on `industry_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *


-- Uncomment the line below when you are ready to run the query, leaving it as your last!

USE LaborStatisticsDB;
GO

-- Join series and industry on industry_code
SELECT TOP 50 *
FROM series AS s
INNER JOIN industry AS i
    ON s.industry_code = i.industry_code
ORDER BY s.series_id;

-- ORDER BY id

## Subqueries, Unions, Derived Tables, Oh My!

1. Write a query that returns the `series_id`, `industry_code`, `industry_name`, and `value` from the `january_2017` table but only if that value is greater than the average value for `annual_2016` of `data_type_code` 82.

In [ ]:

USE LaborStatisticsDB;
GO

SELECT 
    j.series_id,
    s.industry_code,
    i.industry_name,
    j.value
FROM january_2017 AS j
JOIN series AS s
    ON j.series_id = s.series_id
JOIN industry AS i
    ON s.industry_code = i.industry_code
WHERE j.value > (
    SELECT AVG(a.value)
    FROM annual_2016 AS a
    JOIN series AS s2
        ON a.series_id = s2.series_id
    WHERE s2.data_type_code = 82
);



**Optional Bonus Question:** Write the above query as a common table expression!

In [ ]:
-- Optional CTE below

USE LaborStatisticsDB;
GO

-- Common Table Expression version of the subquery
WITH avg_value_cte AS (
    SELECT AVG(a.value) AS avg_value
    FROM annual_2016 AS a
    JOIN series AS s2
        ON a.series_id = s2.series_id
    WHERE s2.data_type_code = 82
)
SELECT 
    j.series_id,
    s.industry_code,
    i.industry_name,
    j.value
FROM january_2017 AS j
JOIN series AS s
    ON j.series_id = s.series_id
JOIN industry AS i
    ON s.industry_code = i.industry_code
CROSS JOIN avg_value_cte
WHERE j.value > avg_value_cte.avg_value;


2. Create a `Union` table comparing average weekly earnings of production and nonsupervisory employees between `annual_2016` and `january_2017` using the data type 30.  Round to the nearest penny.  You should have a column for the average earnings and a column for the year, and the period.

In [ ]:

USE LaborStatisticsDB;
GO

-- Compare average weekly earnings between annual_2016 and january_2017
SELECT 
    ROUND(AVG(a.value), 2) AS avg_weekly_earnings,
    '2016' AS year,
    'Annual' AS period
FROM annual_2016 AS a
JOIN series AS s
    ON a.series_id = s.series_id
WHERE s.data_type_code = 30

UNION

SELECT 
    ROUND(AVG(j.value), 2) AS avg_weekly_earnings,
    '2017' AS year,
    'January' AS period
FROM january_2017 AS j
JOIN series AS s
    ON j.series_id = s.series_id
WHERE s.data_type_code = 30;

## Summarize Your Results

With what you know now about the  Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results and working with the Labor Statistics Database, answer the following questions. Note that while this is subjective, you should include relevant data to back up your opinion.

1. During which time period did production and nonsupervisory employees fare better?

-- Based on the query results, production and nonsupervisory employees fared better in January 2017. 
-- The average weekly earnings increased from $797.20 in 2016 to $808.53 in January 2017, 
-- showing a rise of about 1.4%. This indicates that employees were earning slightly more 
-- at the start of 2017 compared to the previous year.


2. In which industries did production and nonsupervisory employees fare better?

-- Production and nonsupervisory employees fared better in the "Total Private" industry. 
-- Based on the query results, this industry showed the highest values for January 2017, 
-- with average weekly earnings increasing compared to 2016. 
-- This suggests that employees in private sector industries experienced stronger wage growth 
-- than those in other sectors during this period.


3. Now that you have explored the datasets, is there any data or information that you wish you had in this analysis?

-- I wish I had more detailed demographic and regional data, such as age, gender, 
-- education level, and location of employees. This information would help analyze 
-- how wages and employment trends vary across different groups and regions. 
-- Additionally, access to data from more years would allow for stronger trend 
-- analysis and better understanding of long-term economic patterns.
